In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image

# Tensorflow Libraries
from tensorflow import keras
from tensorflow.keras import layers,models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

# System libraries
from pathlib import Path
import os.path

from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = (320, 320)

In [6]:
dataset = "fire_dataset"
image_dir = Path(dataset)

filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

In [7]:
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)

In [8]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [9]:
# Split the data into three categories.
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 640 validated image filenames belonging to 2 classes.
Found 159 validated image filenames belonging to 2 classes.
Found 200 validated image filenames belonging to 2 classes.


In [108]:
cnn= Sequential()
cnn.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(224,224,3)))
cnn.add(MaxPooling2D())
cnn.add(Conv2D(32, (3,3), 1, activation='relu'))
cnn.add(MaxPooling2D())
cnn.add(Conv2D(16, (3,3), 1, activation='relu'))
cnn.add(MaxPooling2D())
cnn.add(Flatten())
cnn.add(Dense(224, activation='relu'))
cnn.add(Dense(2, activation='sigmoid'))

In [109]:
cnn.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [110]:
cnn_hist = cnn.fit(train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=15,)

Epoch 1/15
20/20 [==============================] - 48s 2s/step - loss: 0.5750 - accuracy: 0.7766 - val_loss: 0.2261 - val_accuracy: 0.9434
Epoch 2/15
20/20 [==============================] - 47s 2s/step - loss: 0.2382 - accuracy: 0.9047 - val_loss: 0.1740 - val_accuracy: 0.9560
Epoch 3/15
20/20 [==============================] - 46s 2s/step - loss: 0.1429 - accuracy: 0.9453 - val_loss: 0.1714 - val_accuracy: 0.9371
Epoch 4/15
20/20 [==============================] - 47s 2s/step - loss: 0.1231 - accuracy: 0.9516 - val_loss: 0.1605 - val_accuracy: 0.9434
Epoch 5/15
20/20 [==============================] - 48s 2s/step - loss: 0.0823 - accuracy: 0.9703 - val_loss: 0.1180 - val_accuracy: 0.9434
Epoch 6/15
20/20 [==============================] - 48s 2s/step - loss: 0.0702 - accuracy: 0.9703 - val_loss: 0.1112 - val_accuracy: 0.9434
Epoch 7/15
20/20 [==============================] - 47s 2s/step - loss: 0.0798 - accuracy: 0.9734 - val_loss: 0.1370 - val_accuracy: 0.9560
Epoch 8/15
20/20 [==

In [113]:
cnn_results = cnn.evaluate(test_images, verbose=0)
print("Accuracy: {:.2f}%".format(cnn_results[1] * 100))

Accuracy: 98.00%


In [10]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224,224),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [12]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [16]:
inputs = pretrained_model.input

x = resize_and_rescale(inputs)
x = Dense(256, activation='relu')(pretrained_model.output)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=15,
)



Epoch 1/15
20/20 [==============================] - 60s 3s/step - loss: 0.4126 - accuracy: 0.8016 - val_loss: 0.2270 - val_accuracy: 0.8994
Epoch 2/15
20/20 [==============================] - 56s 3s/step - loss: 0.1540 - accuracy: 0.9531 - val_loss: 0.1666 - val_accuracy: 0.9245
Epoch 3/15
20/20 [==============================] - 58s 3s/step - loss: 0.0772 - accuracy: 0.9797 - val_loss: 0.1386 - val_accuracy: 0.9497
Epoch 4/15
20/20 [==============================] - 58s 3s/step - loss: 0.0644 - accuracy: 0.9859 - val_loss: 0.1354 - val_accuracy: 0.9371
Epoch 5/15
20/20 [==============================] - 55s 3s/step - loss: 0.0523 - accuracy: 0.9875 - val_loss: 0.1283 - val_accuracy: 0.9497
Epoch 6/15
20/20 [==============================] - 55s 3s/step - loss: 0.0401 - accuracy: 0.9859 - val_loss: 0.1257 - val_accuracy: 0.9497
Epoch 7/15
20/20 [==============================] - 55s 3s/step - loss: 0.0388 - accuracy: 0.9922 - val_loss: 0.1237 - val_accuracy: 0.9560
Epoch 8/15
20/20 [==

In [17]:
results = model.evaluate(test_images, verbose=0)
print("Accuracy: {:.2f}%".format(results[1] * 100))

Accuracy: 98.50%


In [114]:
print("Number of layers in the base model: ", len(pretrained_model.layers))
fine_tune_at = 100
for layer in pretrained_model.layers[:fine_tune_at]:
  layer.trainable = False


Number of layers in the base model:  156


In [115]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=Adam(0.0001/10),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [117]:
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    initial_epoch=history.epoch[-1],
    epochs=25,
)

Epoch 15/25
20/20 [==============================] - 48s 2s/step - loss: 0.0101 - accuracy: 0.9984 - val_loss: 0.1206 - val_accuracy: 0.9686
Epoch 16/25
20/20 [==============================] - 34s 2s/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.1169 - val_accuracy: 0.9686
Epoch 17/25
20/20 [==============================] - 32s 2s/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.1152 - val_accuracy: 0.9686
Epoch 18/25
20/20 [==============================] - 35s 2s/step - loss: 0.0075 - accuracy: 0.9984 - val_loss: 0.1192 - val_accuracy: 0.9686
Epoch 19/25
20/20 [==============================] - 34s 2s/step - loss: 0.0065 - accuracy: 0.9984 - val_loss: 0.1223 - val_accuracy: 0.9686
Epoch 20/25
20/20 [==============================] - 32s 2s/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.1266 - val_accuracy: 0.9686
Epoch 21/25
20/20 [==============================] - 31s 2s/step - loss: 0.0059 - accuracy: 0.9984 - val_loss: 0.1297 - val_accuracy: 0.9560
Epoch 22/25
2

In [118]:
results = model.evaluate(test_images, verbose=0)
print("Accuracy: {:.2f}%".format(results[1] * 100))

Accuracy: 99.00%
